In [1]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [2]:
task = 'SMEG'
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[0]
noise_cov = 'baseline_cov'
start = 0.3
stop = 0.5
sfreq = 200
path = op.join(Analysis_path, task, 'meg', 'Stats', str(start)+'_'+str(stop))
filename = op.join(path, name+'-'+noise_cov+'.nc')
test_key = 'N_vs_E+FA-OM'
p_thresh = 0.05

In [8]:
stats = xr.open_dataarray(filename, test_key)
stats.load()

<xarray.DataArray (data: 3, hemisphere: 2, time: 41, src: 10242)>
array([[[[  6.906465e-02, ...,   1.268053e+01],
         ..., 
         [  4.506646e-01, ...,   3.089404e+00]],

        [[  2.904559e-01, ...,   4.491989e+00],
         ..., 
         [  1.215011e+00, ...,   8.858618e-01]]],


       ..., 
       [[[  0.000000e+00, ...,   1.000000e+00],
         ..., 
         [  0.000000e+00, ...,   0.000000e+00]],

        [[  0.000000e+00, ...,   1.300000e+02],
         ..., 
         [  0.000000e+00, ...,   0.000000e+00]]]])
Coordinates:
  * time        (time) float64 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 ...
  * src         (src) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * hemisphere  (hemisphere) object 'left' 'right'
  * data        (data) object 'T_stat' 'p_val' 'clu_inds'

In [6]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [ ]:
if np.where(pv_lh<p_thresh)[0].any():
    t_signif_lh = np.unique(np.where(pv_lh<p_thresh)[0], return_counts=True)
    best_t_lh = t_signif_lh[0][t_signif_lh[1].argsort()][::-1]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

In [ ]:
if np.where(pv_rh<p_thresh)[0].any():
    t_signif_rh = np.unique(np.where(pv_rh<p_thresh)[0], return_counts=True)
    best_t_rh = t_signif_rh[0][t_signif_rh[1].argsort()][::-1]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

In [ ]:
t = int((best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0]))
#t = int(best_t_rh[8])
#t = int(best_t_lh[0])
print('t =', (t+start*sfreq)/sfreq)
mask_lh = np.where(pv_lh[t]<p_thresh)
mask_rh = np.where(pv_rh[t]<p_thresh)
lh = np.where(pv_lh[t]<p_thresh, data_lh[t], 0)
rh = np.where(pv_rh[t]<p_thresh, data_rh[t], 0)

In [ ]:
cmax = np.max(np.abs(np.concatenate([lh, rh])))
clim = (-cmax, cmax)

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both')#, sulcus=True)

#b_obj.add_activation(data=data_lh[t], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_lh[t][mask_lh], vertices=vertices[mask_lh], hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t][mask_rh], vertices=vertices[mask_rh], hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_lh[t]), hide_under=clim[0], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_rh[t]), hide_under=clim[0], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]<0.05, data_lh[t], 0), vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_rh[t]<0.05, data_lh[t], 0), vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
b_obj.add_activation(data=lh, vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
b_obj.add_activation(data=rh, vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)


vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

In [ ]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, (t+start*sfreq)/sfreq))
vb.rotate(custom=(180, -45))
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,test_key+'_t_'+str((t+start*sfreq)/sfreq)+'.png'))

In [ ]:
t_signif_rh